In [1]:
"""Dirichlet-Categorical model.
Posterior inference with Edward's BBVI.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf
from scipy.stats import norm
from time import time

from edward.models import Categorical, Dirichlet, InverseGamma, ParamMixture, Normal, Empirical
import matplotlib.pyplot as plt
plt.style.use('ggplot')

sess = ed.get_session()

sess.run: generate samples
RandomVariable.get_blanket: gets the markov blanket of a distribution

first visualization:
tf.global_variables_initializer().run()
outputs = mus.eval()

### Dirichlet Categorical Model

In [ ]:
N = 1000
K = 4

In [ ]:
# DATA
pi_true = np.random.dirichlet(np.array([20.0, 30.0, 10.0, 10.0]))
z_data = np.array([np.random.choice(K, 1, p=pi_true)[0] for n in range(N)])
print('pi={}'.format(pi_true))

In [ ]:
# MODEL
pi = Dirichlet(tf.ones(4))
z = Categorical(probs=tf.ones([N, 1]) * pi)

In [ ]:
# INFERENCE
qpi = Dirichlet(tf.nn.softplus(tf.Variable(tf.random_normal([K]))))

inference = ed.KLqp({pi: qpi}, data={z: z_data})
inference.run(n_iter=1500, n_samples=30)

In [ ]:
sess = ed.get_session()
print('Inferred pi={}'.format(sess.run(qpi.mean())))

### LDA
Based on: https://github.com/blei-lab/edward/issues/423, https://github.com/blei-lab/edward/pull/588

In [2]:
D = 4 # number of documents
N = [100]*D
K = 10 # number of topics
V = 500 # size of vocabulary

In [3]:
theta = Dirichlet(concentration=tf.zeros([D, K]) + 0.1) # distribution over topics for each document
phi = Dirichlet(concentration=tf.zeros([K, V]) + 0.05)  # distribution over vocab words for each topic

z = [[0] * np.max(N)] * D
w = [[0] * np.max(N)] * D

for d in range(D):
    for n in range(N[d]):
        print("On doc",d,", word", n)
        z[d][n] = Categorical(logits=tf.gather(theta, d))
        w[d][n] = Categorical(logits=tf.gather(phi, z[d][n]))

On doc 0 , word 0
On doc 0 , word 1
On doc 0 , word 2
On doc 0 , word 3
On doc 0 , word 4
On doc 0 , word 5
On doc 0 , word 6
On doc 0 , word 7
On doc 0 , word 8
On doc 0 , word 9
On doc 0 , word 10
On doc 0 , word 11
On doc 0 , word 12
On doc 0 , word 13
On doc 0 , word 14
On doc 0 , word 15
On doc 0 , word 16
On doc 0 , word 17
On doc 0 , word 18
On doc 0 , word 19
On doc 0 , word 20
On doc 0 , word 21
On doc 0 , word 22
On doc 0 , word 23
On doc 0 , word 24
On doc 0 , word 25
On doc 0 , word 26
On doc 0 , word 27
On doc 0 , word 28
On doc 0 , word 29
On doc 0 , word 30
On doc 0 , word 31
On doc 0 , word 32
On doc 0 , word 33
On doc 0 , word 34
On doc 0 , word 35
On doc 0 , word 36
On doc 0 , word 37
On doc 0 , word 38
On doc 0 , word 39
On doc 0 , word 40
On doc 0 , word 41
On doc 0 , word 42
On doc 0 , word 43
On doc 0 , word 44
On doc 0 , word 45
On doc 0 , word 46
On doc 0 , word 47
On doc 0 , word 48
On doc 0 , word 49
On doc 0 , word 50
On doc 0 , word 51
On doc 0 , word 52
On 

### Dynamic Time Model
Original paper: https://mimno.infosci.cornell.edu/info6150/readings/dynamic_topic_models.pdf

### Mixture Model
Based on: https://github.com/blei-lab/edward/blob/master/examples/mixture_gaussian_mh.py, https://github.com/blei-lab/edward/blob/master/examples/mixture_gaussian_gibbs.py

In [ ]:
N = 500  # number of data points
K = 3  # number of components
D = 1  # dimensionality of data
# ed.set_seed(42)

In [ ]:
# DATA
true_mu = np.array([-1.0, 0.0, 1.0], np.float32) * K
true_sigmasq = np.array([1.0**2, 2.0**2, 3.0**2], np.float32)
true_pi = np.array([0.2, 0.3, 0.5], np.float32)

true_z = np.random.choice(np.arange(K), size=N, p=true_pi)
x_data = true_mu[true_z] + np.random.randn(N) * np.sqrt(true_sigmasq[true_z])

In [ ]:
# Prior hyperparameters
pi_alpha = np.ones(K, dtype=np.float32)
mu_sigma = np.std(true_mu)
sigmasq_alpha = 1.0
sigmasq_beta = 2.0

# Model
pi = Dirichlet(pi_alpha)
mu = Normal(0.0, mu_sigma, sample_shape=K)
sigmasq = InverseGamma(sigmasq_alpha, sigmasq_beta, sample_shape=K)
x = ParamMixture(pi, {'loc': mu, 'scale': tf.sqrt(sigmasq)}, Normal,
                 sample_shape=N)
z = x.cat

In [ ]:
# Conditionals
mu_cond = ed.complete_conditional(mu)
sigmasq_cond = ed.complete_conditional(sigmasq)
pi_cond = ed.complete_conditional(pi)
z_cond = ed.complete_conditional(z)

sess = ed.get_session()

# Initialize randomly
pi_est, mu_est, sigmasq_est, z_est = sess.run([pi, mu, sigmasq, z])

In [ ]:
cond_dict = {pi: pi_est, mu: mu_est, sigmasq: sigmasq_est, z: z_est, x: x_data}
t0 = time()
T = 500
for t in range(T):
    z_est = sess.run(z_cond, cond_dict)
    cond_dict[z] = z_est
    pi_est, mu_est = sess.run([pi_cond, mu_cond], cond_dict)
    cond_dict[pi] = pi_est
    cond_dict[mu] = mu_est
    sigmasq_est = sess.run(sigmasq_cond, cond_dict)
    cond_dict[sigmasq] = sigmasq_est
print('took %.3f seconds to run %d iterations' % (time() - t0, T))